# Grouping Data with SQL

## Introduction

In this section, you'll learn how to use aggregate functions in SQL. Because `GROUP BY` collects data into groups, an aggregation function is needed to produce one value represents all the data for each group.


## Objectives

You will be able to:

* Write queries with aggregate functions like `COUNT`, `MAX`, `MIN`, and `SUM`
* Create an alias for the return value of an aggregate function
* Use `GROUP BY` to sort the data sets returned by aggregate functions
* Compare aggregates using the `HAVING` clause

In [3]:
import sqlite3
import pandas as pd

## Database Schema
<img src="Database-Schema.png">

## Connecting to the Database

In [4]:
conn = sqlite3.Connection('data.sqlite')
cur = conn.cursor()

## Groupby and Aggregate Functions

Lets start by looking at some groupby statements to aggregate our data.

In [5]:
#Here we join the offices and employees tables in order to count the number of employees per city.
cur.execute("""SELECT city,
                      COUNT(employeeNumber)
                      FROM offices
                      JOIN employees
                      USING(officeCode)
                      GROUP BY city;""")
pd.DataFrame(cur.fetchall())

,0,1
0,Boston,2
1,London,2
2,NYC,2
3,Paris,5
4,San Francisco,6
5,Sydney,4
6,Tokyo,2


## Ordering and Aliasing
We can also alias our groupby by specifying the number of our selection order that we want to group by. Additionally, we can also order or limit our selection with the order by and limit clauses.

In [6]:
cur.execute("""SELECT city,
                      COUNT(employeeNumber)
                      FROM offices
                      JOIN employees
                      USING(officeCode)
                      GROUP BY 1
                      ORDER BY COUNT(employeeNumber) DESC
                      LIMIT 5;""")
pd.DataFrame(cur.fetchall())
cur.description

(('city', None, None, None, None, None, None),
 ('COUNT(employeeNumber)', None, None, None, None, None, None))

## Retrieving Column Names
* We can get the column names using `.description` attribute of the sqlite3 cursor object (see line 11 in the next cell)

In [7]:
cur.execute("""SELECT city,
                      COUNT(employeeNumber)
                      FROM offices
                      JOIN employees
                      USING(officeCode)
                      GROUP BY 1
                      ORDER BY COUNT(employeeNumber) DESC
                      LIMIT 5;""")
df = pd.DataFrame(cur.fetchall())
# see here:
df.columns = [i[0] for i in cur.description]
df.head()

,city,COUNT(employeeNumber)
0,San Francisco,6
1,Paris,5
2,Sydney,4
3,Boston,2
4,London,2


## Aliasing our Aggregate Function Name
* Now that we can view our column names, we can also practice using alias's to name our aggregations.
* After querying a column, we use the `as` statement before stating a specialized name for the column

In [8]:
cur.execute("""SELECT city,
                      COUNT(employeeNumber) as employeeCount
                      FROM offices
                      JOIN employees
                      USING(officeCode)
                      GROUP BY 1
                      ORDER BY COUNT(employeeNumber) DESC
                      LIMIT 5;""")
df = pd.DataFrame(cur.fetchall())
df. columns = [i[0] for i in cur.description]
df.head()

,city,employeeCount
0,San Francisco,6
1,Paris,5
2,Sydney,4
3,Boston,2
4,London,2


## Other Aggregations

Aside from count() some other useful aggregations include:
    * min()
    * max()
    * sum()
    * avg()

In [9]:
cur.execute("""SELECT customerName,
                      COUNT(*) as number_purchases,
                      MIN(amount) as min_purchase,
                      MAX(amount) as max_purchase,
                      AVG(amount) as avg_purchase,
                      SUM(amount) as total_spent
                      FROM customers
                      JOIN payments
                      USING(customerNumber)
                      GROUP BY 1
                      ORDER BY SUM(amount) DESC;""")
df = pd.DataFrame(cur.fetchall())
df. columns = [i[0] for i in cur.description]
print(len(df))
df.head()

98


,customerName,number_purchases,min_purchase,max_purchase,avg_purchase,total_spent
0,Euro+ Shopping Channel,13,116208.40,65071.26,55056.844615,715738.98
1,Mini Gifts Distributors Ltd.,9,101244.59,85410.87,64909.804444,584188.24
2,"Australian Collectors, Co.",4,44894.74,82261.22,45146.267500,180585.07
3,Muscle Machine Inc,4,20314.44,58841.35,44478.487500,177913.95
4,"Dragon Souveniers, Ltd.",4,105743.00,44380.15,39062.757500,156251.03


In [10]:
df.tail()

,customerName,number_purchases,min_purchase,max_purchase,avg_purchase,total_spent
93,Royale Belge,4,1128.20,1627.56,7304.295000,29217.18
94,Frau da Collezione,2,17746.26,7612.06,12679.160000,25358.32
95,Atelier graphique,3,14571.44,6066.78,7438.120000,22314.36
96,Auto-Moto Classics Inc.,3,5858.56,9658.74,7184.753333,21554.26
97,Boards & Toys Co.,2,3452.75,4465.85,3959.300000,7918.60


 ## The `HAVING` clause vs the `WHERE` clause

Finally, we can also filter our aggregated views with the having clause. The having clause works like the where clause but is used to filter data selections on conditions post the group by. For example, if we wanted to filter based on a customer's last name, we would use the where clause. However, if we wanted to filter a list of city's with at least 5 customers, we would using the having clause; we would first groupby city and count the number of customers, and the having clause allows us to pass conditions on the result of this aggregation.
* `HAVING` clause imposes conditions similar to the where clause, excepts it's used after the data is grouped.
* The difference between the two is that `HAVING` is used AFTER the the `GROUP BY` clause, acting on grouped data, and `WHERE` is used BEFORE the `GROUP BY` clause, acting on data before it has been grouped.

In [11]:
cur.execute("""SELECT city,
                      COUNT(customerNumber) as number_customers
                      FROM customers
                      GROUP BY 1
                      HAVING COUNT(customerNumber)>=5;""")
df = pd.DataFrame(cur.fetchall())
df. columns = [i[0] for i in cur.description]
print(len(df))
df.head()

2


,city,number_customers
0,Madrid,5
1,NYC,5


## Combining the where and having clause
We can also use the where and having clause in conjunction with each other for more complex rules.
For example, let's say we want a list of customers who have made at least 3 purchases of over 50K each.

In [12]:
cur.execute("""SELECT customerName,
                      COUNT(amount) as number_purchases_over_50K
                      FROM customers
                      JOIN payments
                      USING(customerNumber)
                      WHERE amount >= 50000
                      GROUP BY 1
                      HAVING COUNT(amount) >= 3
                      ORDER BY COUNT(amount) DESC;""")
df = pd.DataFrame(cur.fetchall())
df. columns = [i[0] for i in cur.description]
print(len(df))
df.head()

53


,customerName,number_purchases_over_50K
0,Euro+ Shopping Channel,13
1,Mini Gifts Distributors Ltd.,9
2,"Anna's Decorations, Ltd",4
3,"Australian Collectors, Co.",4
4,Baane Mini Imports,4


In [13]:
df.tail()

,customerName,number_purchases_over_50K
48,"Stylish Desk Decors, Co.",3
49,Suominen Souveniers,3
50,"Toys of Finland, Co.",3
51,Toys4GrownUps.com,3
52,Vitachrome Inc.,3


## Summary

After this section, you should have a good idea of how to use aggregate functions, aliases and the having clause to filter selections.

In [14]:
cur.execute('''SELECT * from customers LIMIT 25;''')
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df
cur.description

(('customerNumber', None, None, None, None, None, None),
 ('customerName', None, None, None, None, None, None),
 ('contactLastName', None, None, None, None, None, None),
 ('contactFirstName', None, None, None, None, None, None),
 ('phone', None, None, None, None, None, None),
 ('addressLine1', None, None, None, None, None, None),
 ('addressLine2', None, None, None, None, None, None),
 ('city', None, None, None, None, None, None),
 ('state', None, None, None, None, None, None),
 ('postalCode', None, None, None, None, None, None),
 ('country', None, None, None, None, None, None),
 ('salesRepEmployeeNumber', None, None, None, None, None, None),
 ('creditLimit', None, None, None, None, None, None))

In [15]:
cur.execute("""SELECT city,
                      COUNT(customerNumber) as number_customers
                      FROM customers
                      GROUP BY 1
                      ORDER BY number_customers desc;""")
df = pd.DataFrame(cur.fetchall())
df. columns = [i[0] for i in cur.description]
print(len(df))
df.head(20)

96


,city,number_customers
0,Madrid,5
1,NYC,5
2,Brickhaven,3
3,Paris,3
4,Singapore,3
5,Auckland,2
6,Boston,2
7,Cambridge,2
8,Frankfurt,2
9,Glendale,2


In [16]:
cur.execute("""SELECT city,
                      COUNT(customerNumber) as number_customers
                      FROM customers
                      GROUP BY 1
                      HAVING COUNT(customerNumber)>=5;""")
df = pd.DataFrame(cur.fetchall())
df. columns = [i[0] for i in cur.description]
print(len(df))
df.head()

2


,city,number_customers
0,Madrid,5
1,NYC,5


In [20]:
cur.execute("""
SELECT customerName, city, creditLimit
                      COUNT(amount) as number_purchases_over_50K
                      FROM customers
                      JOIN payments
                      USING(customerNumber)
                      WHERE amount >= 50000
                      GROUP BY customerName
                      HAVING COUNT(amount) >= 3
                      ORDER BY COUNT(amount) DESC;
""")
df = pd.DataFrame(cur.fetchall())
df. columns = [i[0] for i in cur.description]
print(len(df))
df.head()

OperationalError: near "(": syntax error

In [18]:
<p style="page-break-after:always;"></p>

SyntaxError: invalid syntax (<ipython-input-18-df87d080efd4>, line 1)